# Covid-19 Dashboard

In [2]:
import ipywidgets as wdg
import pandas as pd
import matplotlib.pyplot as plt
import json
import time
from uk_covid19 import Cov19API

In [3]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [4]:
# Load initial data from disk¶
#Loading initial data from testsbydate.json file. 
#The file has datewise data of number of Planned covid tests (PlanCovidTests), number of Test-1, 
#Test-2 and Test-3 types carried on the date marked as testsone, 
#teststwo and teststhree respectivelyin the UK.
with open("testsbydate.json", "rt") as INFILE:
    data=json.load(INFILE)
INFILE.close()


In [5]:
#function to parse date in date string format using pandas
def parse_date(datestring):
    return pd.to_datetime(datestring, format = "%Y-%m-%d")
## Wrangle the data (Function)
#1. prints the data update date
#2. makes dataframe (df) against the json data on ascending date basis
#3. There is a new column in df TotalCovidTests which sums all three types of tests (test1, test2 and test3) which is total tests carried all over UK on a given date.
#4. This is called when "Refresh Data" button is pressed
def wrangle_data(rawdata):
    datalist = rawdata['data']
    update = rawdata['lastUpdate']
    global update1
    update1 = parse_date(update)
    dtime.value = str(update1)
    dates = [dictionary['date'] for dictionary in datalist]
    dates.sort()
    
    startdate = parse_date(dates[0])
    enddate = parse_date(dates[-1])
    
    index = pd.date_range(startdate, enddate, freq = 'D')
    df = pd.DataFrame(index=index, columns=['PlanCovidTests', 'testsone', 'teststwo', 'teststhree'])
    
    for entry in datalist: # each entry is a dictionary with date, cases, hospital and deaths
        date=parse_date(entry['date'])
        for column in ['PlanCovidTests', 'testsone', 'teststwo', 'teststhree']:
            if pd.isna(df.loc[date, column]):
                value = int(entry[column]) if entry[column] != None else 0;
                df.loc[date, column] = value

    df.fillna(0, inplace=True)
    df['TotalCovidTests'] = df['testsone'] + df['teststwo'] + df['teststhree']

    return df


In [6]:
## Download current data (function)
#1. function to download current data - The function tries to call api from PHE, 
#  if it fails it just copies the data from the  
#2. This is called when "Refresh Data" button is pressed.
#function to call current data
def access_api():
    filters = [
        'areaType=overview'
    ]

    structure = {
        "date" : "date",
        "PlanCovidTests" : "newTestsByPublishDate",
        "testsone" : "newPillarOneTestsByPublishDate",
        "teststwo" : "newPillarTwoTestsByPublishDate",
        "teststhree" : "newPillarThreeTestsByPublishDate"
    }
    try:
        api = Cov19API(filters = filters, structure = structure)
        testdata = api.get_json()
    except: 
        print("No new data can be downloaded at the moment")
        print("Presenting as per old data")       
        with open("testsbydate.json", "rt") as INFILE:
            testdata=json.load(INFILE)
        INFILE.close()
    
    return testdata

In [7]:
## Functions to refresh graph, button call back for data refresh
#New data will be downloaded and graph is updated if the "Refresh Data" button is pressed, 
#see the changes on graph after pressing the refresh button.
#when the refresh data button is pressed
def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=scale.value
    if current==scale.options[0][1]:
        other=scale.options[1][1]
    else:
        other=scale.options[0][1]
    scale.value=other # forces the redraw
    scale.value=current # now we can change it back
   
    #Refresh data button to download new data and update the graph
def api_button_callback(button):
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global df
    df=wrangle_data(apidata)
    refresh_graph()
    button.icon="check"
    time.sleep(4)
    button.icon = "download"

# button widget
button = wdg.Button(
    description = 'Update Data',
    disabled = False,
    button_style = '',
    tooltip = 'Click to update data',
    icon = 'download'
)

In [8]:
## Graphs and Analysis
## Graphs and Analysis
#Graph is the comparison between Planned Covid tests (PlanCovidTests) Vs 
#  Actual total Covid tests (TotalCovidTests) carried out on datewise basis.
#Graph is the comparison between Planned Covid tests (PlanCovidTests) Vs
#  Actual total Covid tests (TotalCovidTests) carried out on datewise basis.
def plot_graph(test, scale):
    df[list(test)].plot(logy = scale)

#widget for linear or log
scale=wdg.Dropdown(
    options=[('linear', False),('log', True)],
    value= False,
    description='Scale: ',
    disabled=False,
)
#widget for total testes and planned to be tested
Tests = wdg.SelectMultiple(
    options = ['PlanCovidTests', 'TotalCovidTests'],
    value = ['PlanCovidTests', 'TotalCovidTests'],
    row = 2,
    description = "Tests:",
    disabled = False
)
#widget for displaying updated date
say = "hello"
dtime = wdg.Text(
    value= say,
    placeholder="",
    description='Data Date:',
    disabled=True
)



## A comparison of the number of test carried out (TotalCovidTests) with planned testing capacity(PlanCovidTests)
There are two interactive widgets
1. Tests - multiple selection can be done here - Where either select PlanCovidTests or TotalCovidTests or both can be selected.
2. Scale - Single selection can only be done - Where the selection is either linear or log, depends on the way the graph can se seen.

In [9]:
# Deploying the Dashboard
df=wrangle_data(data)
graph=wdg.interactive_output(plot_graph,{'test' : Tests,'scale' : scale})
button.on_click(api_button_callback) # the name of your function inside these brackets
control = wdg.HBox([button, Tests, scale])
display(control, graph)
display(dtime)

Output()

Text(value='2020-11-24 19:41:32+00:00', description='Data Date:', disabled=True, placeholder='')

**Author and Copyright Notice** Rohit Garg (r.garg@se20.qmul.ac.uk). All Rights Reserved. Data source: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*